In [2]:
import pandas as pd
import geopandas as gpd

In [56]:
wiki = pd.read_csv("wikipedia.csv")
osm = gpd.read_file("../osm/code/tourism_vancouver.geojson")

osm_rename = osm.rename(columns={"name": "title"})


In [57]:
osm_wiki = osm_rename.join(wiki.set_index('title'), on='title')

In [61]:
# Filter out by "uminteresting" tourist features #
# ################################################

# I've decided to keep all: museum, attraction, gallery, theme_park, train_car, Botanical_garden and aquarium features
# Meanwhile excluding hotels, viewpoints and artwork based on if there is a wikipedia article for that feature

#List of features that we are interested in

tourism_list = ['museum', 'attraction', 'gallery', 'hotel', 'viewpoint', 'artwork', 'theme_park', 'train_car', 'Botanical_garden', 'aquarium']
osm_wiki_filtered = osm_wiki[osm_wiki.tourism.isin(tourism_list)]

# Remove any features which don't contain a title
osm_wiki_filter_notitle = osm_wiki_filtered[osm_wiki_filtered.title.notnull()]

# Identify features demmed "uninteresting" by virtue of having a wikipedia page
# Amount of interest can be gauged by number of words contained in wikipedia article and can be adjusted accordingly
wordcount_threshold = 1

#hotels
osm_wiki_bad_hotels = osm_wiki_filter_notitle[(osm_wiki_filter_notitle.tourism == 'hotel') & ( (osm_wiki_filter_notitle.content.str.count(' ') < wordcount_threshold) |(osm_wiki_filter_notitle.content.isnull()) )] 

#viewpoints
osm_wiki_bad_viewpoints = osm_wiki_filter_notitle[(osm_wiki_filter_notitle.tourism == 'viewpoint') & ( (osm_wiki_filter_notitle.content.str.count(' ') < wordcount_threshold) |(osm_wiki_filter_notitle.content.isnull()) )] 

#artwork
osm_wiki_bad_artwork = osm_wiki_filter_notitle[(osm_wiki_filter_notitle.tourism == 'artwork') & ( (osm_wiki_filter_notitle.content.str.count(' ') < wordcount_threshold) |(osm_wiki_filter_notitle.content.isnull()) )] 

In [67]:
# Remove the identified "uninteresting" features from dataframe
osm_wiki_filter_hotels = osm_wiki_filter_notitle.drop(osm_wiki_bad_hotels.index)
osm_wiki_filter_viewpoints = osm_wiki_filter_hotels.drop(osm_wiki_bad_viewpoints.index)
osm_wiki_final = osm_wiki_filter_viewpoints.drop(osm_wiki_bad_artwork.index)


In [69]:
osm_wiki_final.to_file("best_destinations_vancouver.geojson", driver='GeoJSON')
